# Sentiment Analysis using FinBERT

## Modules 

In [1]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

<ipython-input-1-ce2e9558c130>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [2]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [3]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [4]:
# Clean the cl_path
# try:
#     shutil.rmtree(cl_path) 
# except:
#     pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at /Users/zengxin/Study/Econ2355/finBERT-master/models/language_model/finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceCla

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [5]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
# finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [10]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

04/14/2021 14:01:10 - INFO - finbert.finbert -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False


## Fine-tune the model

In [11]:
# Get the training examples
train_data = finbert.get_data('train')

In [12]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [13]:
# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [14]:
trained_model = finbert.train(train_examples = train_data, model = model)

04/14/2021 14:01:24 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:01:24 - INFO - finbert.utils -   guid: train-1
04/14/2021 14:01:24 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
04/14/2021 14:01:24 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:01:24 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:01:24 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/

04/14/2021 14:08:36 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:08:36 - INFO - finbert.utils -   guid: validation-1
04/14/2021 14:08:36 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/14/2021 14:08:36 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:08:36 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:08:36 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:08:36 - INFO - finbert.utils -   label: neutral (id = 2)
04/14/2021 14:08:36 - INFO - finbert.finbert -   ***** Loading data *****
04/14/2


Validation losses: [0.7006283952639654]
No best model found


Epoch:  25%|██▌       | 1/4 [07:47<23:22, 467.60s/it]

04/14/2021 14:19:50 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:19:50 - INFO - finbert.utils -   guid: validation-1
04/14/2021 14:19:50 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/14/2021 14:19:50 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:19:50 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:19:50 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:19:50 - INFO - finbert.utils -   label: neutral (id = 2)
04/14/2021 14:19:50 - INFO - finbert.finbert -   ***** Loading data *****
04/14/2

...


Validation losses: [0.7006283952639654, 0.38051716639445377, 0.34192835367642915]


Epoch:  75%|███████▌  | 3/4 [32:59<10:22, 622.22s/it]

04/14/2021 14:49:41 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:49:41 - INFO - finbert.utils -   guid: validation-1
04/14/2021 14:49:41 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/14/2021 14:49:41 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:49:41 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:49:41 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:49:41 - INFO - finbert.utils -   label: neutral (id = 2)
04/14/2021 14:49:41 - INFO - finbert.finbert -   ***** Loading data *****
04/14/2


Validation losses: [0.7006283952639654, 0.38051716639445377, 0.34192835367642915, 0.3386951134755061]


Epoch: 100%|██████████| 4/4 [48:49<00:00, 732.28s/it]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [15]:
test_data = finbert.get_data('test')

In [16]:
results = finbert.evaluate(examples=test_data, model=trained_model)

04/14/2021 14:50:15 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:50:15 - INFO - finbert.utils -   guid: test-1
04/14/2021 14:50:15 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
04/14/2021 14:50:15 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:50:15 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:50:15 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:50:15 - IN

### Prepare the classification report

In [17]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [18]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [19]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.39
Accuracy:0.82

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.85      0.77       267
           1       0.75      0.91      0.82       128
           2       0.92      0.79      0.85       575

    accuracy                           0.82       970
   macro avg       0.79      0.85      0.82       970
weighted avg       0.84      0.82      0.82       970



### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [20]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [21]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/zengxin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
result = predict(text,model)

04/14/2021 14:51:57 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:51:57 - INFO - finbert.utils -   guid: 0
04/14/2021 14:51:57 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
04/14/2021 14:51:57 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:51:57 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:51:57 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [24]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.01936759, 0.95101064, 0.029621843]",negative,-0.931643,0.051746
1,The news rapidly infected financial markets.,"[0.039290782, 0.9071576, 0.053551633]",negative,-0.867867,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.0087142065, 0.98210055, 0.009185263]",negative,-0.973386,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.041827083, 0.95001715, 0.008155786]",negative,-0.908190,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.009766736, 0.98195595, 0.008277356]",negative,-0.972189,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.017705781, 0.964261, 0.018033177]",negative,-0.946555,0.000000


In [25]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.93.


Here is another example

In [26]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [27]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

04/14/2021 14:52:10 - INFO - finbert.utils -   *** Example ***
04/14/2021 14:52:10 - INFO - finbert.utils -   guid: 0
04/14/2021 14:52:10 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
04/14/2021 14:52:10 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:52:10 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 14:52:10 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [28]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.9362365, 0.037263386, 0.026500136]",positive,0.898973,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.02644885, 0.010628035, 0.96292317]",neutral,0.015821,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.15177582, 0.006117092, 0.8421071]",neutral,0.145659,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.9253749, 0.019313268, 0.055311784]",positive,0.906062,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.84090143, 0.014810475, 0.14428812]",positive,0.826091,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.06258039, 0.008866058, 0.9285535]",neutral,0.053714,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.7201432, 0.0125278905, 0.26732892]",positive,0.707615,0.000000


In [29]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.51.


### Then, we calculate the sentiment score for all our OCR files

In [38]:
import pandas as pd
results = pd.DataFrame()
source_dir = '/Users/zengxin/Study/Econ2355/OCR_English'
for dirpath, dirnames, filenames in os.walk(source_dir):
    for subfile in filenames:
        file_name = os.path.join(source_file, subfile)
        with open(file_name, encoding="utf8") as f:
            text = f.readlines()
        if len(text) == 0:
            continue
        result = predict(text[0], model)
        blob = TextBlob(text[0])
        result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
        result['file_name'] = subfile
        result = result.drop(["sentence"], axis=1)
        results = pd.concat([results, result], axis=0)
        print(results)

04/14/2021 15:16:43 - INFO - finbert.utils -   *** Example ***
04/14/2021 15:16:43 - INFO - finbert.utils -   guid: 0
04/14/2021 15:16:43 - INFO - finbert.utils -   tokens: [CLS] event : in 2018 , the company achieved total operating income of 56 . 63 shareholders of the parent company was 4 . 03 ##6 billion yuan , a decrease of about 6 . 89 % over the same period last year ; earnings per share was 0 . 30 yuan , a decrease of 0 . 02 yuan compared with 2017 [SEP]
04/14/2021 15:16:43 - INFO - finbert.utils -   input_ids: 101 2724 1024 1999 2760 1010 1996 2194 4719 2561 4082 3318 1997 5179 1012 6191 15337 1997 1996 6687 2194 2001 1018 1012 6021 2575 4551 11237 1010 1037 9885 1997 2055 1020 1012 6486 1003 2058 1996 2168 2558 2197 2095 1025 16565 2566 3745 2001 1014 1012 2382 11237 1010 1037 9885 1997 1014 1012 6185 11237 4102 2007 2418 102
04/14/2021 15:16:43 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

                                    logit prediction  sentiment_score  \
0  [0.015611071, 0.9772495, 0.0071394513]  negative  -0.961638          

   textblob_prediction              file_name  
0 -0.10303              600023-20190422-0.txt  


04/14/2021 15:16:57 - INFO - finbert.utils -   *** Example ***
04/14/2021 15:16:57 - INFO - finbert.utils -   guid: 0
04/14/2021 15:16:57 - INFO - finbert.utils -   tokens: [CLS] the performance in the third quarter has recovered and continues to improve in the later period [SEP]
04/14/2021 15:16:57 - INFO - finbert.utils -   input_ids: 101 1996 2836 1999 1996 2353 4284 2038 6757 1998 4247 2000 5335 1999 1996 2101 2558 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:16:57 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:16:57 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:16:57 - INFO - finbert.utils -   label: None (id = 9090)
04/14/2021 15:16:57 - INFO -

                                    logit prediction  sentiment_score  \
0  [0.015611071, 0.9772495, 0.0071394513]  negative  -0.961638          
0  [0.92903036, 0.049480543, 0.021489102]  positive   0.879550          

   textblob_prediction              file_name  
0 -0.10303              600023-20190422-0.txt  
0  0.00000              600028-20201028-0.txt  


04/14/2021 15:17:13 - INFO - finbert.utils -   *** Example ***
04/14/2021 15:17:13 - INFO - finbert.utils -   guid: 0
04/14/2021 15:17:13 - INFO - finbert.utils -   tokens: [CLS] 1 ##h ' s operating performance is beautiful , and the net profit at ##tri ##bu ##table to the parent increases at the same time [SEP]
04/14/2021 15:17:13 - INFO - finbert.utils -   input_ids: 101 1015 2232 1005 1055 4082 2836 2003 3376 1010 1998 1996 5658 5618 2012 18886 8569 10880 2000 1996 6687 7457 2012 1996 2168 2051 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:17:13 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:17:13 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/14/2021 15:17:13 - INFO - finbert.

...

                                     logit prediction  sentiment_score  \
0   [0.015611071, 0.9772495, 0.0071394513]  negative  -0.961638          
0   [0.92903036, 0.049480543, 0.021489102]  positive   0.879550          
0   [0.91511595, 0.060164854, 0.024719158]  positive   0.854951          
0   [0.95304894, 0.02486723, 0.022083867]   positive   0.928182          
1   [0.2370496, 0.75452715, 0.008423247]    negative  -0.517478          
..                                   ...         ...        ...          
0   [0.28860012, 0.027875189, 0.68352467]   neutral    0.260725          
0   [0.06794674, 0.02179613, 0.9102571]     neutral    0.046151          
0   [0.050118905, 0.012111436, 0.9377696]   neutral    0.038007          
0   [0.05906313, 0.013624487, 0.9273124]    neutral    0.045439          
0   [0.060288586, 0.01381705, 0.92589444]   neutral    0.046472          

    textblob_prediction              file_name  
0  -0.103030             600023-20190422-0.txt  
0   0.000000 

04/14/2021 15:46:09 - INFO - finbert.utils -   *** Example ***
04/14/2021 15:46:09 - INFO - finbert.utils -   guid: 0
04/14/2021 15:46:09 - INFO - finbert.utils -   tokens: [CLS] [UNK] [UNK] [UNK] 高 [UNK] [UNK] [UNK] 的 , [UNK] [UNK] [UNK] [UNK] “ [UNK] [UNK] 2018 / 1 / 1 ) , [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 。 [UNK] [UNK] dc ##f 和 p ##b [UNK] [UNK] 目 [UNK] [UNK] 4 . 1 - 4 . 4 元 , [UNK] [UNK] [UNK] [UNK] “ [UNK] [UNK] ” [UNK] [UNK] 。 [SEP]
04/14/2021 15:46:09 - INFO - finbert.utils -   input_ids: 101 100 100 100 1981 100 100 100 1916 1010 100 100 100 100 1523 100 100 2760 1013 1015 1013 1015 1007 1010 100 100 100 100 100 100 1636 100 100 5887 2546 1796 1052 2497 100 100 1918 100 100 1018 1012 1015 1011 1018 1012 1018 1769 1010 100 100 100 100 1523 100 100 1524 100 100 1636 102
04/14/2021 15:46:09 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
04/14/2021 15:46:09 

                                     logit prediction  sentiment_score  \
0   [0.015611071, 0.9772495, 0.0071394513]  negative  -0.961638          
0   [0.92903036, 0.049480543, 0.021489102]  positive   0.879550          
0   [0.91511595, 0.060164854, 0.024719158]  positive   0.854951          
0   [0.95304894, 0.02486723, 0.022083867]   positive   0.928182          
1   [0.2370496, 0.75452715, 0.008423247]    negative  -0.517478          
..                                   ...         ...        ...          
0   [0.06794674, 0.02179613, 0.9102571]     neutral    0.046151          
0   [0.050118905, 0.012111436, 0.9377696]   neutral    0.038007          
0   [0.05906313, 0.013624487, 0.9273124]    neutral    0.045439          
0   [0.060288586, 0.01381705, 0.92589444]   neutral    0.046472          
0   [0.056080014, 0.014900665, 0.92901933]  neutral    0.041179          

    textblob_prediction              file_name  
0  -0.103030             600023-20190422-0.txt  
0   0.000000 

04/14/2021 15:46:22 - INFO - finbert.utils -   *** Example ***
04/14/2021 15:46:22 - INFO - finbert.utils -   guid: 0
04/14/2021 15:46:22 - INFO - finbert.utils -   tokens: [CLS] 三 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 。 公 司 [UNK] [UNK] 2020 [UNK] [UNK] [UNK] , [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 的 方 法 , [UNK] 原 [UNK] 的 年 [UNK] 平 [UNK] 法 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 法 。 [UNK] [UNK] 2020 年 中 [UNK] , [UNK] 会 [UNK] [UNK] [SEP]
04/14/2021 15:46:22 - INFO - finbert.utils -   input_ids: 101 1741 100 100 100 100 100 100 100 100 1636 1772 1791 100 100 12609 100 100 100 1010 100 100 100 100 100 100 100 100 100 100 100 1916 1863 1901 1010 100 1787 100 1916 1840 100 1839 100 1901 100 100 100 100 100 100 1901 1636 100 100 12609 1840 1746 100 1010 100 1763 100 100 102
04/14/2021 15:46:22 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

                                     logit prediction  sentiment_score  \
0   [0.015611071, 0.9772495, 0.0071394513]  negative  -0.961638          
0   [0.92903036, 0.049480543, 0.021489102]  positive   0.879550          
0   [0.91511595, 0.060164854, 0.024719158]  positive   0.854951          
0   [0.95304894, 0.02486723, 0.022083867]   positive   0.928182          
1   [0.2370496, 0.75452715, 0.008423247]    negative  -0.517478          
..                                   ...         ...        ...          
0   [0.050118905, 0.012111436, 0.9377696]   neutral    0.038007          
0   [0.05906313, 0.013624487, 0.9273124]    neutral    0.045439          
0   [0.060288586, 0.01381705, 0.92589444]   neutral    0.046472          
0   [0.056080014, 0.014900665, 0.92901933]  neutral    0.041179          
0   [0.112235926, 0.016505225, 0.8712588]   neutral    0.095731          

    textblob_prediction              file_name  
0  -0.103030             600023-20190422-0.txt  
0   0.000000 

In [39]:
results.to_csv('/Users/zengxin/Study/Econ2355/result/sentiment_finbert.csv')